# Script to match the relevant tables for 4Hi-q

## Requirements

To be able to run this, the following requirements need to be met:

- A `python 3.10` environment, including the `astropy`, `astroquery`, `scipy` and standard modules
- The `function package` and the `catalogues` in the same directory as this script.

## Structure

This script consists of a few separate parts to handle the catalogue data, divided into the following sections:

- Querying for the catalogue data and writing backups to disk, requiring the following:
  - A `region` specified in the beginning, used for RA and DEC constraints
  - An `agn_cat`, providing RA, DEC and probability information on the AGNs
  - The `sweep_cat` catalogues for the region in question (TODO: Use the LS DR query interface)
  - The `vhs_cat` is obtained by querying the Vista Science Archive
- Matching the catalogues, also writing a backup.
  - The matching is performed on the respective RA and DEC coordinates (TODO: Use the advanced constraints described in my thesis)
  - A match via the `cds` service is performed with the GALEX catalogue
- Processing the catalogues (unit conversions and reddening correction), including a split into *pointlike* and *extended*.
- Collapsing everything into a `LePhare` input file
- Running `LePhare`:
  - Collect the filter information
  - Collect the template information
  - Run the photo-z-routine

## Citations

- Please see `other/citations.md` for any citations/acknowledgements that might be required when using this script.

All of the functions used for these routines can either be found directly in this notebook or in the `function_package` directory.

In [ ]:
import os
import function_package as fp
import logging
logging.getLogger().setLevel("INFO")

logging.info("Modules loaded successfully")

In [ ]:
# Set up the directories necessary
fp.generate_all_filepaths()

In [ ]:
# Define the region to constrain everything to:
# The eFEDs field is dec < 6.2 & dec > -3.2 & ra < 146.2 & ra > 126.
ra_min, ra_max, dec_min, dec_max = 125, 130, -5, 0
REGION = fp.Region(ra_min, ra_max, dec_min, dec_max)
logging.info(REGION)

## Loading the tables

In this section, the tables are loaded.\
TODO: Make use of astroquery if possible


In [ ]:
SHU_TABLE = fp.load_and_clean_opt_agn_shu(REGION)

In [ ]:
VHS_TABLE = fp.load_and_clean_vhs(REGION)

In [ ]:
SWEEP_TABLE = fp.load_and_clean_sweep(REGION)

## Matching

Now that the tables are properly loaded (except for GALEX, which we'll acquire while matching), we can match them one by one.\
TODO: Implement the functions if possible

In [24]:
# In my master thesis, I found the following to be a reasonable matching radius between sweep and the shu et al. agn table:
match_radius_shu = 0.1
MATCH_TABLE = fp.match_shu_with_sweep(SWEEP_TABLE, SHU_TABLE, match_radius=match_radius_shu)

INFO:root:Match of agn to sweep successful;
For 0 of the 1899 sources in the agn table, no match was found within the given radius.
INFO:root:From now on, the ra and dec columns refer to the sweep ra and dec.


In [34]:
# In my master thesis, I found the following to be a reasonable matching radius to vhs:
match_radius_vhs = 0.19
MATCH_TABLE2 = fp.match_vhs_to_table(MATCH_TABLE, VHS_TABLE, match_radius=match_radius_vhs)

INFO:root:Found 683 matching vhs sources within the prescribed radius.


In [ ]:
# In my master thesis, I found the following to be a reasonable matching radius to galex:
match_radius_galex = 2.1
MATCH_TABLE3 = fp.match_with_galex_and_clean_it(MATCH_TABLE2, match_radius=match_radius_galex)

In [38]:
# Write a backup of the matched table to disk
fp.write_table_as_backup(MATCH_TABLE3, "match_backup")

INFO:root:Successfully written a match_backup file at c:\Users\fabia\OneDrive\Dokumente\Studium\Master\Thesis\LePhare_work\new_attempt/data/match_backups/base_full_match.fits.


## Processing

The matched table can now be processed.\
For this, all columns are turned into corrected magnitude ones, and the table is split up into a *pointlike* and an *extended* part.

In [40]:
MATCH_TABLE3 = fp.read_table_from_backup("match_backup")
PROCESSED = fp.process_galex_columns(MATCH_TABLE3)
PROCESSED = fp.process_sweep_columns(PROCESSED)
POINTLIKE, EXTENDED = fp.split_table_by_sourcetype(PROCESSED)
POINTLIKE = fp.process_vhs_columns(POINTLIKE)
EXTENDED = fp.process_vhs_columns(EXTENDED)
fp.write_table_as_backup(POINTLIKE, "processed_backup", "pointlike")
fp.write_table_as_backup(EXTENDED, "processed_backup", "extended")

KeyError: 'flux_fuv'

## Create LePhare input files

Now that we do have the pointlike and extended subsets, we can create the LePhare input table:

In [ ]:
POINTLIKE_LEPHARE = fp.process_for_lephare(POINTLIKE)
EXTENDED_LEPHARE = fp.process_for_lephare(EXTENDED)
fp.write_table_as_backup(POINTLIKE_LEPHARE, "lephare_in", "pointlike", overwrite=True)
fp.write_table_as_backup(EXTENDED_LEPHARE, "lephare_in", "extended", overwrite=True)

## Run LePhare

Before we run any of the subprograms, we need to initialise important paths:

In [ ]:
FILTER_STEM = "base"
PARA_STEM = "base"
INPUT_STEM = "base"

PATH_PARA_IN = fp.get_filepath("para_in")
PATH_FILTER_REPO = fp.get_filepath("filter_repo")
PATH_FILTER_FILE = fp.get_filepath("filter", FILTER_STEM)
PATH_LEPHAREDIR = fp.get_filepath("lepharedir")


### 1) Run the filter file

This step can be skipped if the filterfile has already been set up before.

In [ ]:
!{$fpath_lephare}/source/filter -c $fpath_para \
    -FILTER_REP $fpath_filter_repo \
    -FILTER_FILE $filter_stem \
    >$fpath_filter_file

### 2) Run the template files

This step can be skipped if the required template files already exist.

### 3) Run the photo-z routine

This is the core functionality of LePhare.\
Be sure you know what you're doing when modifying the parameters.